In [2]:
import pandas as pd
import numpy as np
import itertools as it
import time

In [24]:
df = pd.read_csv('frequencies.csv')

# df_master = pd.read_feather('/Users/anujgautam/Documents/Research-workspace/MESA-tests/megasaurus.feather')


In [16]:
top = df_master.query('m==1.7 & v==4 & z==0.015').iloc[10]
model_to_modes = []
for n in np.arange(1,11,1):
    for l in np.arange(4):
        for m in np.arange(-1*l,l+1,1):
            if m==0:
                freq = top[f"n{n}ell{l}m0"]
            else:
                freq = top[f"n{n}ell{l}m0"] + m*top[f"n{n}ell{l}dfreq"]
            nlm = f"{n}{l}{m}"
            n_pg = n
            if l==0:
                n_p=n_pg
            else:
                n_p=n_pg-1
            n_g=0
            model_to_modes.append([nlm,freq,l,m,n_g,n_p,n_pg])

            
rot_freqs = pd.DataFrame(data=model_to_modes,columns=["nlm","freq","l","m","n_g","n_p","n_pg"])
rot_freqs = rot_freqs.query("l<=1").dropna()
freqs = rot_freqs["freq"].values

In [17]:
rot_freqs

,nlm,freq,l,m,n_g,n_p,n_pg
0,100,8.032503,0,0,0,1,1
16,200,11.385054,0,0,0,2,2
17,21-1,11.892394,1,-1,0,1,2
18,210,12.089676,1,0,0,1,2
19,211,12.286958,1,1,0,1,2
32,300,15.026756,0,0,0,3,3
33,31-1,15.806892,1,-1,0,2,3
34,310,16.005165,1,0,0,2,3
35,311,16.203438,1,1,0,2,3
48,400,18.507386,0,0,0,4,4


In [18]:
time_start = time.time()
factor_limit = 3
for x in it.combinations(range(-factor_limit, factor_limit+1), factor_limit):
    pass
time_end = time.time()

print(time_end-time_start)

8.821487426757812e-05


In [19]:
def factors_set(factor_limit):
    """ generator of factors i, j, k
        Parameters
        -------
        factor_limit : int
            limit of the absolute value of factors before each term

        Yields
        -------
        result : <class 'numpy.ndarray'>
            array of 3 factors in order [i j k]."""
    l = []
    for i in np.arange(-1*factor_limit, factor_limit+1, 1):
        for j in np.arange(-1*factor_limit, factor_limit+1, 1):
            for k in np.arange(-1*factor_limit, factor_limit+1, 1):
                    l.append([i, j, k])
    return l

In [ ]:
def linear_combination(value, f, resolution=0, factor_limit=3):
    """ calculates linear combination satisfying:
        value = i*f1 + j*f2 + k*f3 within certain resolution
        
        Parameters
        ----------
        value : float
            value to be represented by linear combination
        f : <class 'numpy.ndarray'>
            array of 3 terms to construct linear combination
        resolution : float, default=0
            maximal differance to accept combination as solution
        factor_limit : int, default=3
            limit of the absolute value of factors before each term

        Returns
        -------
        result : <class 'numpy.ndarray'>
            array of 3 factors [i j k] of the combination 
            or None if fails to find one."""
            
    result = None
    for factors in it.product(range(-factor_limit, factor_limit+1), repeat=factor_limit):
        factors = np.array(factors)
        _sum = 0
        # _sum = np.sum(factors*f)
        for i in range(len(factors)):
            _sum += factors[i] * f[i]
        if abs(_sum - value) <= resolution:
            # check if it's first combination,
            # later prioritise single term combinations (harmonics) and 
            # ones with smaller absolute sum of factors
            if result is None or (sum(abs(factors)) <= sum(abs(result)) and
                    np.count_nonzero(factors) <= np.count_nonzero(result)):
                result = factors
    return result




df.sort_values(by=['amp'], ascending=False, inplace=True)
df = df.query('amp>0.1')
# df = rot_freqs
n = 0
results = []
factor_limit = 2
i = 0
for freq, combos in it.product([df['freq'].iloc[n]], it.combinations(df['freq'][n+1:], 2)):
    result = linear_combination(freq, combos, 0.037, factor_limit=factor_limit) 
    if result is not None:
        results.append([combos, result])
        i+=1
print(i)

In [36]:
i = 0
df['freq'].iloc[0], np.sum(np.array(results[i][0])*np.array(results[i][1]))

(21.747407864081463, 21.770360240855766)

In [58]:
factors

(2, 2)

In [25]:
freqs = df.sort_values(by='amp', ascending=False)['freq']

In [26]:
threshold = 0.037
factor_limit = 2
i = 0
res = []
for combo in it.combinations(freqs, 2):
    for factors in it.product(range(-factor_limit, factor_limit+1), repeat=factor_limit):
        if not ((0, 1) == factors or (1, 0) == factors):
            for freq in freqs:
                if abs(freq - (factors[0]*combo[0] + factors[1]*combo[1])) < threshold:
                    i+=1
                    res.append([combo[0], combo[1], freq, factors[0], factors[1]])
result = pd.DataFrame(res, columns=['f1', 'f2', 'f_result', 'factor1', 'factor2'])
result.drop_duplicates(subset=['f_result'], keep='first', inplace=True)
result = result[~result['f_result'].isin(result['f1']) | ~result['f_result'].isin(result['f2'])].reset_index(drop=True)

In [31]:
result.sort_values(by='f_result')

,f1,f2,f_result,factor1,factor2
15,52.244814,17.738660,16.746377,1,-2
6,21.747408,30.404965,17.281310,-2,2
13,52.244814,34.514368,17.738660,1,-1
30,41.491586,23.547330,17.953700,1,-1
34,24.579642,43.426363,18.863305,-1,1
32,24.579642,45.079211,20.462622,-1,1
19,44.848690,23.078329,21.747408,1,-1
17,44.848690,33.503583,22.165077,-1,2
41,27.467266,50.450241,22.967963,-1,1
0,21.747408,44.848690,23.078329,-1,1


In [30]:
len(freqs)

57